# Statistical Inference for Means
In this Notebook, we will work on confidence intervals and statistical inference for means. This particular Notebook is mostly adopted from the [Inferential Statistics](https://www.coursera.org/learn/inferential-statistics-intro/home/welcome) course of Duke University, converted from R to Python and tweaked to match the needs of our CSMODEL course.

Our Notebooks in CSMODEL are designed to be guided learning activities. To use them, simply through the cells from top to bottom, following the directions along the way. If you find any unclear parts or mistakes in the Notebooks, email your instructor.

## Instructions
* Read each cell and implement the TODOs sequentially. The markdown/text cells also contain instructions which you need to follow to get the whole notebook working.
* Do not change the variable names unless the instructor allows you to.
* Answer all the markdown/text cells with 'Question #' on them. Write your answer in the Canvas Quiz.
* You are expected to search how to some functions work on the Internet or via the docs. 
* The notebooks will undergo a 'Restart and Run All' command, so make sure that your code is working properly.
* You are expected to understand the dataset loading and processing separately from this class.
* You may not reproduce this notebook or share them to anyone.

## Import Libraries

For the statistical functions, we will be using `scipy`, specifically, the `stats` submodule. The [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) module provides a number of probability distribution functions, summary and frequency statistics, correlation functions, statistical tests, and more.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import ttest_ind

## Real Estate Data

Let's consider the real estate data from the city of Ames, Iowa. The details of every real estate transaction in Ames is recorded by the City Assessor's  office. Our particular focus will be all residential home sales in Ames between 2006 and 2010.  This collection represents our **population** of interest. We would like to learn about these home sales by taking smaller samples from the full population. Let's load the data.

In [ ]:
ames_df = pd.read_csv('ames.csv', index_col='Order')
ames_df.head()

Call the [`info()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) function.

In [ ]:
ames_df.info()

Our dataset contains 81 variables and 2930 instances.

### Get a Sample

Here, we have access to the population data. But in most cases, we do not. Instead, we have to work with a **sample**. Let's try to take a sample from our population using the [`sample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) function.

**Note**: The random state is any number that allows us to make our notebooks reproducible. The random state, in very simple terms, dictates where to start "searching" and sampling at random.

In [ ]:
n = 60
ames_sample_df = ames_df.sample(n, random_state=8)
ames_sample_df.head()

For now, we will only focus on the `Lot.Area` variable. Let us compute the summary statistics for this variable.

In [ ]:
agg = ames_sample_df.agg({'Lot.Area': ['mean', 'median', 'std']})

sample_mean = agg.loc['mean'][0]
sample_median = agg.loc['median'][0]
sample_std = agg.loc['std'][0]

print('Sample Mean: {:.2f}'.format(sample_mean))
print('Sample Median: {:.2f}'.format(sample_median))
print('Sample Standard Deviation: {:.2f}'.format(sample_std))

**Question #1:** What is the mean of your sample? Limit to 2 decimal places.

Sample Mean: 9494.48


### Confidence Interval

Based on this sample, what can we infer about the population? Based only on this single sample, the best estimate of the average living area of houses sold in Ames would be the sample mean, usually denoted as $\bar{x}$. That serves as a good point estimate but it would be useful to also communicate how uncertain we are of that estimate. This uncertainty can be quantified using a confidence interval.

A confidence interval for a population mean is computed as:

$$\bar{x} \pm ME $$

where $ME$ is also known as the **margin of error**. The margin of error is computed as:

$$ME = z^* \times \frac{s}{\sqrt{n}}$$


Where $z^*$, also known as the **critical value**.

Confidence level corresponds to the probability of getting a value within the confidence interval when sampling is repeated multiple times. On the other hand, the significance level or alpha ($\alpha$) is the probability of rejecting the null hypothesis given that the null hypothesis is true. These two have an inverse relationship, where the significance level is equivalent to $(1 - CL)$, where $CL$ is the confidence level.

To get the value of $z^*$, we can use the [`norm.ppf()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) function. A 95% confidence level refers to the middle 95% of the distribution. Thus, your significance level is 0.05. Suppose that we are performing a one-tailed test, the critical value for this is computed as:

In [ ]:
alpha = 0.05
z_star_95_one_tailed = norm.ppf(1 - alpha)
print('{:.2f}'.format(z_star_95_one_tailed))

When performing a two-tailed test, the value of $\alpha$ should be divided into two. You may read more about this [here](https://thedatascientist.com/comparing-significance-level-confidence-level-and-confidence-interval/). Thus, we compute $z^*$ as:

In [ ]:
alpha = 0.05
z_star_95_two_tailed = norm.ppf(1 - alpha / 2)
print('{:.2f}'.format(z_star_95_two_tailed))

Let's assume that we are performing a two-tailed test. Compute and display the margin of error given a 95% confidence level. Use the variables from previous cells instead of using the actual values.

In [ ]:
# Write your code here
margin_of_error = z_star_95_two_tailed * (sample_std / np.sqrt(n))
print('{:.2f}'.format(margin_of_error))

**Question #2:** Given a 95% confidence level, what is the margin of error? Limit to 2 decimal places.

Margin of error: 1082.47

Again, the 95% confidence interval is the sample mean $\pm$ the margin of error. Compute and display the minimum and maximum values in the 95% confidence interval. Use the variables from previous cells instead of using the actual values.

In [ ]:
# Write your code here
minimum_value = sample_mean - margin_of_error
maximum_value = sample_mean + margin_of_error
print('Minimum Value: {:.2f}'.format(minimum_value))
print('Maximum Value: {:.2f}'.format(maximum_value))

**Question #3:** What is the minimum value of the 95% confidence interval? Limit to 2 decimal places.

Minimum Value: 8412.01

**Question #4:** What is the maximum value of the 95% confidence interval? Limit to 2 decimal places.

Maximum Value: 10576.95

To recap: even though we don't know what the full population looks like, we believe that the true average size of houses in Ames lies between the lower and upper values 95% of the time. There are a few conditions that must be met for this interval to be valid. These conditions are:

- Samples must be independent
- Sample size must be at least 30 (or population is normally distributed)

### Verify if Our Confidence Interval Covers the True Mean

In this case, we have the rare luxury of knowing the true population mean since we have data on the entire population. Let's calculate this value so that we can determine if our confidence intervals actually capture it.

Let us get the mean from the population (not the sample).

Compute and display the true population mean for the variable.

In [ ]:
# Write your code here
population_mean = ames_df['Lot.Area'].mean()
print('{:.2f}'.format(population_mean))

**Question #5:** What is the true population mean of the variable? Limit to 2 decimal places.

Poulation Mean: 10147.92

**Note:** The true population mean should be within your computed confidence interval range.

### Increase the Confidence Level to 99%

Let's get another sample from the population, where `n` is 60.

In [ ]:
n = 60
ames_sample_df = ames_df.sample(n, random_state=9)
ames_sample_df.head()

Let's focus on the `Lot.Area` variable again.

Compute and display the summary statistics - mean, median, and standard deviation for this variable.

In [ ]:
# Write your code here
agg = ames_sample_df.agg({'Lot.Area': ['mean', 'median', 'std']})

sample_mean = agg.loc['mean'][0]
sample_median = agg.loc['median'][0]
sample_std = agg.loc['std'][0]

print('Sample Mean: {:.2f}'.format(sample_mean))
print('Sample Median: {:.2f}'.format(sample_median))
print('Sample Standard Deviation: {:.2f}'.format(sample_std))

**Question #6:** What is the mean of your new sample? Limit to 2 decimal places.

Sample Mean: 9137.33


Now, let's increase the confidence level from 95% to 99% and perform a two-tailed test. Get the value of $z^*$ or the z-score that corresponds to the middle 99% of the data.

In [ ]:
# Write your code here
alpha = 0.01
z_star_99_two_tailed = norm.ppf(1 - alpha / 2)
print('{:.2f}'.format(z_star_99_two_tailed))


Compute and display the margin of error. Use the variables from previous cells instead of using the actual values.

In [ ]:
# Write your code here
margin_of_error = z_star_99_two_tailed * (sample_std / np.sqrt(n))
print('{:.2f}'.format(margin_of_error))

**Question #7:** Given a 99% confidence level, what is the margin of error? Limit to 2 decimal places.

Margin of error: 2097.74

Compute and display the minimum and maximum values in the 99% confidence interval. Use the variables from previous cells instead of using the actual values.

In [ ]:
# Write your code here
minimum_value = sample_mean - margin_of_error
maximum_value = sample_mean + margin_of_error
print('Minimum Value: {:.2f}'.format(minimum_value))
print('Maximum Value: {:.2f}'.format(maximum_value))


**Question #8:** What is the minimum value of the 99% confidence interval? Limit to 2 decimal places.

Minimum Value: 7039.60

**Question #9:** What is the maximum value of the 99% confidence interval? Limit to 2 decimal places.

Maximum Value: 11235.07

**Note:** The true population mean should be within your computed confidence interval range.

From here, we have seen that even though we do not have access to the population, we can use a sample to estimate the the true population mean with the use of confidence intervals.

## Birth Records Data

In 2004, the state of North Carolina released a large data set containing information on births recorded in this state. This data set is useful to researchers studying the relation between habits and practices of expectant mothers and the birth of their children. We will work with a random sample of observations from this data set.

Load the `nc` data set into our workspace.

In [ ]:
nc_df = pd.read_csv('nc.csv')
nc_df.head()

We have observations on 13 different variables, some categorical and some numerical. The meaning of each variable is as follows.

- **`fage`**: father’s age in years.
- **`mage`**:	mother’s age in years.
- **`mature`**: maturity status of mother.
- **`weeks`**: length of pregnancy in weeks.
- **`premie`**: whether the birth was classified as premature (premie) or full-term.
- **`visits`**: number of hospital visits during pregnancy.
- **`marital`**: whether mother is married or not married at birth.
- **`gained`**: weight gained by mother during pregnancy in pounds.
- **`weight`**: weight of the baby at birth in pounds.
- **`lowbirthweight`**: whether baby was classified as low birthweight (low) or not (not low).
- **`gender`**: gender of the baby, female or male.
- **`habit`**: status of the mother as a nonsmoker or a smoker.
- **`whitemom`**:	whether mom is white or not white.

We will consider the possible relationship between a mother's smoking habit (`habit`) and the weight (`weight`) of her baby. Plotting the data is a useful first step because it helps us quickly visualize trends, identify strong associations, and develop research questions.

Let's use a boxplot to compare the two groups:

In [ ]:
nc_df.groupby('habit').boxplot(column='weight')
plt.show()

Now let's look at the summary statistics across the two groups.

In [ ]:
summary_stat = nc_df.groupby('habit').agg({'weight': ['mean', 'median', 'std', len]})
summary_stat

It appears that babies of smokers tend to have less weight, but is this difference statistically significant? In order to answer this question, we will conduct a hypothesis test.

### Hypothesis Test

Based on the our sample, the difference in the means of the baby weights for smokers and non-smokers is:

In [ ]:
non_smoker_mean = summary_stat.loc['nonsmoker'].loc['weight'].loc['mean']
smoker_mean = summary_stat.loc['smoker'].loc['weight'].loc['mean']

diff = non_smoker_mean - smoker_mean
print('{:.2f}'.format(diff))

We set up our hypotheses as follows:

$H_0$ (null hypothesis): The true difference is 0.

$H_A$ (alternative hypothesis): The true difference is not 0.

Now, we can use a $t$-test to compare the two means from the unpaired groups. We'll use the [`ttest_ind()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) function. This function assumes that the null hypothesis is that the difference between the two means is 0, while the alternative hypothesis is that the diference between them is not 0. We set the `equal_var` parameter to `False` because we don't want to assume that the population has equal variances.

In [ ]:
ttest_ind(nc_df[nc_df['habit'] == 'smoker']['weight'],
          nc_df[nc_df['habit'] == 'nonsmoker']['weight'],
          equal_var = False)

Note that you the function above is to perform a $t$-test for **independent means** (unpaired). We would need to use other functions if we need to perform tests for other groups. We leave this for you to find out.

**Question #10:** What can you conclude based on the $p$-value under a 5% significance level? Do we accept or reject the null hypothesis?

**Question #11:** Can we say that smoking among mothers causes their babies to be lighter?

## 90% Confidence Interval

Compute the **90%** confidence interval for the average baby weights using the `nc` dataset.

Compute and display the sample mean. No need to sample -- use the whole dataset.

In [ ]:
# Write your code here
agg = nc_df.agg({'weight': ['mean', 'median', 'std']})

sample_mean = agg.loc['mean'][0]
sample_median = agg.loc['median'][0]
sample_std = agg.loc['std'][0]

print('Sample Mean: {:.2f}'.format(sample_mean))
print('Sample Median: {:.2f}'.format(sample_median))
print('Sample Standard Deviation: {:.2f}'.format(sample_std))

Compute and display the minimum and maximum values in the 90% confidence interval. Use the variables from previous cells instead of using the actual values.

In [ ]:
# Write your code here
alpha = 0.10
z_star_90_two_tailed = norm.ppf(1 - alpha / 2)
print('Z: {:.2f}'.format(z_star_90_two_tailed))

margin_of_error = z_star_90_two_tailed * (sample_std / np.sqrt(n))
print('Margin of error: {:.2f}'.format(margin_of_error))

minimum_value = sample_mean - margin_of_error
maximum_value = sample_mean + margin_of_error
print('Minimum Value: {:.2f}'.format(minimum_value))
print('Maximum Value: {:.2f}'.format(maximum_value))


**Question #12:** Is the sample mean within the confidence interval? Yes or No?

Yes